# Setup

In [1]:
!pip install -q modelscan
!modelscan -v

modelscan, version 0.5.0


In [2]:
!pip install -q xgboost==1.7.6
!pip install -U -q scikit-learn==1.3.0

In [3]:
import pickle
from pathlib import Path
import os
import numpy as np
from utils.pickle_codeinjection import generate_unsafe_file
from utils.xgboost_diabetes_model import train_model, get_predictions

# Save a XGBoost Model

The model is trained on a diabetes dataset, and predicts whether a person has diabetes or not. The dataset can be found here: [Link to PIMA Indian diabetes dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database). The model is saved at ```./XGBoostModels/safe_model.pkl```

In [4]:
model_directory = os.path.join(os.getcwd(), "XGBoostModels")
if not os.path.isdir(model_directory):
    os.mkdir(model_directory)

safe_model_path_pickle = os.path.join(model_directory, "safe_model.pkl")
model = train_model()
with open(safe_model_path_pickle, "wb") as fo:
    pickle.dump(model, fo)

# Predict using Safe Model

In [5]:
number_of_predictions = 3
get_predictions(number_of_predictions, model)

The model predicts: [0, 1, 1]
The true labels are: [0. 1. 1.]


# Scan the safe model

The scan results include information on the files scanned, and any issues if found. For the safe model scanned, modelscan finds no code injections in it, as expected.

In [6]:
!modelscan -p XGBoostModels/safe_model.pkl

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/safe_model.pkl using modelscan.scanners.PickleUnsafeOpScan model scan

--- Summary ---

 No issues found! 🎉


# Model Serialization Attack

Here code is injected in the safe model to read aws secret keys. The unsafe model is saved at ```./XGBoostModels/unsafe_model.pkl```

In [7]:
# Inject code with the command
command = "system"
malicious_code = """cat ~/.aws/secrets
    """

In [8]:
with open(safe_model_path_pickle, "rb") as fo:
    safe_model_pickle = pickle.load(fo)

unsafe_model_path = os.path.join(model_directory, "unsafe_model.pkl")
generate_unsafe_file(model, command, malicious_code, unsafe_model_path)

# Predict using Unsafe Model

The malicious code gets executed when the model is loaded. The aws secret keys are displayed. 

Also, the unsafe model predicts just as well as safe model i.e., the code injection attack will not impact the model performance. The unaffected performance of unsafe models makes the ML models an effective attack vector. 

In [9]:
with open(unsafe_model_path, "rb") as fo:
    unsafe_model = pickle.load(fo)

get_predictions(number_of_predictions, unsafe_model)

aws_access_key_id=<access_key_id>
aws_secret_access_key=<aws_secret_key>
The model predicts: [0, 1, 1]
The true labels are: [0. 1. 1.]


# Scan the Unsafe Model

The scan results include information on the files scanned, and any issues if found. In this case, a critical severity level issue is found in the unsafe model scanned. 

modelscan also outlines the found operator(s) and module(s) deemed unsafe. 

In [10]:
!modelscan -p XGBoostModels/unsafe_model.pkl

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/unsafe_model.pkl using modelscan.scanners.PickleUnsafeOpScan model scan

--- Summary ---

Total Issues: 1

Total Issues By Severity:

    - LOW: 0
    - MEDIUM: 0
    - HIGH: 0
    - CRITICAL: 1

--- Issues by Severity ---

--- CRITICAL ---

Unsafe operator found:
  - Severity: CRITICAL
  - Description: Use of unsafe operator 'system' from module 'posix'
  - Source: /Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/unsafe_model.pkl


# Reporting Format
ModelScan can report scan results in console (default), JSON, or custom report (to be defined by user in settings-file). For mode details, please see:  ` modelscan -h` 

## JSON Report

For JSON reporting: `modelscan -p ./path-to/file -r json -o output-file-name.json` 

In [11]:
# This will save the scan results in file: xgboost-model-scan-results.json
!modelscan --path  XGBoostModels/unsafe_model.pkl -r json -o xgboost-model-scan-results.json

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/unsafe_model.pkl using modelscan.scanners.PickleUnsafeOpScan model scan
{"modelscan_version": "0.5.0", "timestamp": "2024-01-25T17:56:00.855056", 
"input_path": 
"/Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/unsafe_model.pkl
", "total_issues": 1, "summary": {"total_issues_by_severity": {"LOW": 0, 
"MEDIUM": 0, "HIGH": 0, "CRITICAL": 1}}, "issues_by_severity": {"CRITICAL": 
[{"description": "Use of unsafe operator 'system' from module 'posix'", 
"operator": "system", "module": "posix", "source": 
"/Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/unsafe_model.pkl
", "scanner": "modelscan.scanners.PickleUnsafeOpScan"}]}, "errors": [], 
"scanned": {"total_scanned": 1, "scanned_files": 
["/Users/mehrinkiani/Documents/modelscan/notebooks/XGBoostModels/unsafe_model.pk
l"]